In [ ]:
from pyspark.sql import SparkSession
import requests
import pandas as pd
import io
import logging

# Iniciar a Spark Session
spark = SparkSession.builder \
    .appName("Ler Parquet e Escrever no PostgreSQL") \
    .getOrCreate()


In [ ]:
# Configuração do Logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Criar um logger
logger = logging.getLogger(__name__)

logger.info("Criando sessão Spark....")

In [ ]:
logger.info("Reading Postgres Authentication Params. Please wait.")
# Ler o arquivo Parquet
df_unifiedData_gold = spark.read.parquet("/FileStore/glassdoor/gold/glassdoor_tb")
try: 
    url = "https://sqliteonline.com/#sharedb=p%3Abig"
    table = "public.glassdoor_grupo1_atividade3"
    user = "postgres"
    password = "root"

    df_unifiedData_gold.printSchema()

except Exception as e:
    logger.error(f"Error!! Não foi possível ler parametros de autenticação do Postgres: {e}")

root
 |-- trimestre: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- cnpj_if: string (nullable = true)
 |-- instituicao_financeira: string (nullable = true)
 |-- indice: string (nullable = true)
 |-- quantidade_de_reclamacoes_reguladas_procedentes: string (nullable = true)
 |-- quantidade_de_reclamacoes_reguladas_outras: string (nullable = true)
 |-- quantidade_de_reclamacoes_nao_reguladas: string (nullable = true)
 |-- quantidade_total_de_reclamacoes: string (nullable = true)
 |-- quantidade_total_de_clientes_ccs_e_scr: string (nullable = true)
 |-- quantidade_de_cliente_ccs: string (nullable = true)
 |-- quantidade_de_cliente_scr: string (nullable = true)
 |-- employer_website: string (nullable = true)
 |-- employer_headquarters: string (nullable = true)
 |-- employer_founded: string (nullable = true)
 |-- employer_industry: string (nullable = true)
 |-- employer_revenue: string (nullable = true)
 |-- geral: string (nullable 

In [ ]:
logger.info("PostgreSQL setup and properties. Please wait while processing...")
# Ler o arquivo Parquet
df_unifiedData_gold = spark.read.parquet("/FileStore/glassdoor/gold/glassdoor_tb")
try: 
    # Configurar o ambiente Spark para usar o PostgreSQL
    spark = SparkSession.builder \
        .appName("Write DataFrame to PostgreSQL") \
        .config("spark.jars.packages", "org.postgresql:postgresql:42.2.5") \
        .getOrCreate()

    # Corrigir a URL de conexão
    url = "jdbc:postgresql://localhost:5432/glassdoor_db"

    # Configurações para a conexão JDBC
    properties = {
        "user": "postgres",
        "password": "root",
        "driver": "org.postgresql.Driver"
    }
    logger.info(df_unifiedData_gold.printSchema())
    logger.info("Configuração do PostgreSQL realizada com sucesso!")
    logger.info("Tentativa de realizar o import dos dados no PostgreSQL. Aguarde....")
    # Tentativa de salvar o DataFrame no PostgreSQL
    try:
        df_unifiedData_gold.write \
            .jdbc(url=url, table="public.glassdoor_grupo1_atividade3", mode="overwrite", properties=properties)
        print("Dados salvos com sucesso no PostgreSQL.")
        
    except Exception as e:
        print(f"ERROR - Erro ao salvar os dados no PostgreSQL: {e}")
        
except Exception as e:
    logger.error(f"ERROR - Não foi possível realizar configuração do PostgreSQL: {e}")




Erro ao salvar os dados: An error occurred while calling o1811.jdbc.
: org.postgresql.util.PSQLException: Connection to localhost:5432 refused. Check that the hostname and port are correct and that the postmaster is accepting TCP/IP connections.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:319)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:49)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:223)
	at org.postgresql.Driver.makeConnection(Driver.java:400)
	at org.postgresql.Driver.connect(Driver.java:259)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:123)
	at org.apache.spark.sql.jdbc.JdbcD